In [1]:
# Import packages
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from statsmodels.tsa.seasonal import STL
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime as dt
from matplotlib.colors import TwoSlopeNorm
import time
from rasterstats import zonal_stats
from osgeo import gdal
import re

In [2]:
# This function rescales the pixel value such that we derive the physical values
def rescale(j, nodata_value_new, Scaling, Offset, file_path):
    tiff_open = gdal.Open(file_path) # open the GeoTIFF we want to rescale
    band = tiff_open.GetRasterBand(1) # Select the band
    tiff_array = tiff_open.ReadAsArray() # Assign raster values to a numpy nd array

    nodata_value_old = tiff_open.GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
    #nodata_value_new = -9999 # new data value

    tiff_array_new = np.where(tiff_array != nodata_value_old, tiff_array * Scaling + Offset, nodata_value_new) # Rescale pixel values and change nodatavalue to -9999

    col = tiff_open.RasterXSize # number of columns
    rows = tiff_open.RasterYSize # number of rows
    driver = tiff_open.GetDriver()

    # Create a new raster to save the average-values
    new_raster = driver.Create(j, col, rows, 1, gdal.GDT_Float32)

    # Copy the properties of the original raster
    new_raster.SetGeoTransform(tiff_open.GetGeoTransform())
    new_raster.SetProjection(tiff_open.GetProjection())

    # Add the average values to newly created raster
    new_raster.GetRasterBand(1).WriteArray(tiff_array_new)
    new_raster.GetRasterBand(1).SetNoDataValue(nodata_value_new)

    # close the raster
    new_raster = None
    del new_raster

Before running the code, we should set Offset and Scaling equal to as provided on the website. Furthermore, create the folders to which we will save the rescaling GeoTIFFs.

In [5]:
offset = 0
scaling = 1 / 30
ndv = -9999

# For each of the LAI images, we rescale the Geotiff
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/Copernicus/LAI/Telangana/aoi/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/mieke/Documents/Msc Thesis/Datasets/Copernicus/LAI/Telangana/aoi_rescaled/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    rescale(i, ndv, scaling, offset, tiff_path)

Below, we find the exact same approach used for Karnataka.

In [6]:
offset = 0
scaling = 1 / 30
ndv = -9999

# For each of the LAI images, we rescale the Geotiff
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/Copernicus/LAI/Karnataka/aoi/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/mieke/Documents/Msc Thesis/Datasets/Copernicus/LAI/Karnataka/aoi_rescaled/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    rescale(i, ndv, scaling, offset, tiff_path)

The rescaling is finished. Below is just a small check.

In [7]:
path_r = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/Copernicus/LAI/Karnataka/aoi_rescaled/' + i # the path of the selected GeoTIFF
LAI_r = rasterio.open(path_r, mode = 'r') # open the selected GeoTIFF
LAI_r_array = LAI_r.read(1) # Assign raster values to a numpy nd array
row_r_134 = LAI_r_array[134]
print(row_r_134[357])
type(row_r_134[357])
LAI_r.nodata

-9999.0


-9999.0

Below are some elder versions of the for loop (as on DiCRA).

In [4]:
Offset = 0
Scaling = 1 / 30

folder_names = os.listdir('C:/Users/mieke/Documents/Msc Thesis/Datasets/DiCRA/LAI') # contains all foldernames within the folder NDVI_masked
for i in folder_names:
    file_names = os.listdir('C:/Users/mieke/Documents/Msc Thesis/Datasets/DiCRA/LAI/' + i + '/RASTER') # contains all filesnames within the folder selected folder i
    os.chdir('C:/Users/mieke/Documents/Msc Thesis/Datasets/DiCRA/LAI_rescaled/' + i) # Set path
    for j in file_names:        
        tiff_path = "C:/Users/mieke/Documents/Msc Thesis/Datasets/DiCRA/LAI/" + i + "/RASTER/" + j # the selected GeoTIFF path
        rescale(j, tiff_path)

In [ ]:
Offset = 0
Scaling = 1 / 30

# For each of the LAI images, we rescale the Geotiff
folder_names = os.listdir('C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/LAI/Input/LAI images') # contains all foldernames
os.chdir('C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/LAI/Output/LAI_rescaled/') # Set path
for i in folder_names:
    file_names = os.listdir('C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/LAI/Input/LAI images/' + i) # contains all filesnames 
    
    for j in file_names:
        LAI_data = "C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/LAI/Input/LAI images/" + i + "/" + j # the selected GeoTIFF path
        rescale(j, tiff_path)